# Phonon computations

This is a quick sketch how to run a simple phonon calculation using DFTK.
First we run an SCF calculation.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(bulk(:Si); pseudopotentials, functionals=LDA())
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.916448349404                   -0.69    4.8    104ms
  2   -7.921208691179       -2.32       -1.52    1.0   52.6ms
  3   -7.921404522876       -3.71       -2.49    1.4   80.7ms
  4   -7.921440843063       -4.44       -2.82    2.8   75.2ms
  5   -7.921441596832       -6.12       -2.98    1.1   54.2ms
  6   -7.921442006636       -6.39       -4.51    1.0   53.1ms
  7   -7.921442021746       -7.82       -4.56    2.9   92.2ms
  8   -7.921442022126       -9.42       -5.18    1.0   53.9ms
  9   -7.921442022143      -10.79       -6.05    1.4   58.4ms
 10   -7.921442022144      -11.98       -5.85    2.4   71.1ms
 11   -7.921442022144      -12.38       -6.46    1.0   58.2ms
 12   -7.921442022144      -13.64       -7.19    1.0   54.9ms
 13   -7.921442022144      -14.45       -7.55    1.5   61.6ms
 14   -7.921442022144   +  -14.75       -8.38    1.1   59.6ms


Next we compute the phonon modes at the q-point `[1/4, 1/4, 1/4]`.

In [2]:
scfres = DFTK.unfold_bz(scfres)
phret_q0 = @time DFTK.phonon_modes(scfres; q=[0.25, 0.25, 0.25]);
nothing  # hide

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.1   2.77s  Non-interacting
   1        0       1        0.31     52.8   18.7s  
   2        0       2       -0.52     47.2   1.91s  
   3        0       3       -2.40     41.8   1.38s  
   4        0       4       -4.46     29.9   1.07s  
   5        0       5       -6.46     17.5   756ms  
   6        0       6       -8.92      4.0   420ms  
   7        1       1       -7.85     55.7   2.11s  Restart
   8        1       2       -9.03      5.6   459ms  
                                      72.9   2.03s  Final orbitals
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   1.84s  Non-interacting
   1        0       1        0.31     52.8   2.09s  
   2        0       2       -0.52     47.2   1.53s  
   3

These are the final phonon frequencies:

In [3]:
phret_q0.frequencies

6-element Vector{Float64}:
 -0.0034981553634571673
 -0.002977451164477944
 -0.0029774511631113164
  0.004302230617776421
  0.00430223061860389
  0.0043532017506919115

> **Preliminary implementation**
>
> Practical phonon computations have only seen rudimentary testing as of now.
> As of now we do not yet recommend relying on this feature for production
> calculations. We appreciate any issues, bug reports or PRs.